In [1]:
import gc
from all_stand_var import conv_dict, lab_cols, used_cols
from all_own_funct import memory_downscale,memory_upscale,evaluate,lin_reg_coef
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from seaborn import heatmap
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import kurtosis
from matplotlib.backends.backend_pdf import PdfPages
import keras.backend as K
from RNN_LTSM_CHD import return_loaded_model
import pickle
import locale
import LR_build_CHD as pp
import tables
import datetime as dt


from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

locale.setlocale(locale.LC_ALL, 'fr_FR')

# output folder in which al results are saved
output_folder = os.path.join(os.getcwd(), 'Results_LR_RF_CHD_v3','dynamic')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import tables
# Read in data
df=pd.read_hdf(os.path.join('./Results_CHD_v5/final', 'processed_df.h5'),key='df',mode='r')
#df=memory_downscale(df)
print(df.info())



<class 'pandas.core.frame.DataFrame'>
MultiIndex: 978473 entries, (0, 7171037, Timestamp('2008-04-11 00:00:00')) to (77, 3536787, Timestamp('2020-01-10 00:00:00'))
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   pat_datetime     978473 non-null  datetime64[ns]
 1   pat_weight_act   978473 non-null  float16       
 2   mon_etco2        829237 non-null  float16       
 3   mon_hr           971134 non-null  float16       
 4   mon_ibp_mean     887213 non-null  float16       
 5   mon_rr           931288 non-null  float16       
 6   mon_sat          959128 non-null  float16       
 7   vent_m_fio2      973620 non-null  float16       
 8   vent_m_peep      960680 non-null  float16       
 9   vent_m_ppeak     977208 non-null  float16       
 10  vent_m_rr        978466 non-null  float16       
 11  vent_m_tv_exp    972433 non-null  float16       
 12  Age              978473 non-null  float16 

In [3]:
# calculate the hour intervals, and also the missingness parameter

df=df.reset_index(drop=False)

def index_1hr(group):
    if len(group) > 720:
        group=group.sort_values('pat_datetime')
        group=group.tail(720)
   
    group['tim']=pd.date_range(start='1/1/2018', periods=len(group), freq='1min')
    grouped = pd.Grouper(key='tim', freq='1H')
    group['idx_1hr']=group.groupby(grouped,sort=False).ngroup().add(1)
    group['mis']=(group['pat_datetime'].max()-group['pat_datetime'].min()).total_seconds()/60
    del group['tim']
    return group

df=df.groupby('Adnum',sort=False,as_index=False).apply(index_1hr)
df['idx_1hr']=df['idx_1hr'].astype(str)
print(df['idx_1hr'].unique())
df[['mis']]=StandardScaler().fit_transform(df[['mis']])
df=df.set_index(['pat_hosp_id','OK_datum'])
df=df.sort_values('idx_1hr',ascending=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978473 entries, 0 to 978472
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   level_0          978473 non-null  int64         
 1   pat_hosp_id      978473 non-null  int64         
 2   OK_datum         978473 non-null  datetime64[ns]
 3   pat_datetime     978473 non-null  datetime64[ns]
 4   pat_weight_act   978473 non-null  float16       
 5   mon_etco2        829237 non-null  float16       
 6   mon_hr           971134 non-null  float16       
 7   mon_ibp_mean     887213 non-null  float16       
 8   mon_rr           931288 non-null  float16       
 9   mon_sat          959128 non-null  float16       
 10  vent_m_fio2      973620 non-null  float16       
 11  vent_m_peep      960680 non-null  float16       
 12  vent_m_ppeak     977208 non-null  float16       
 13  vent_m_rr        978466 non-null  float16       
 14  vent_m_tv_exp    972

In [4]:
# split dataframe into train, test and validation set, stratified
pat=pd.read_excel(r'Results_CHD\admissiondate_CHD0.xlsx',
                    parse_dates=['OK_datum'],index_col=0)
group=pat.groupby('pat_hosp_id',sort=False).max().reset_index()
group.drop_duplicates('pat_hosp_id',inplace=True)

df_train,df_val,df_test=pp.split_stratified_into_train_val_test(group, stratify_colname='Reintub',
                                         frac_train=0.7, frac_val=0.1, frac_test=0.2,
                                         random_state=1)

train_pat=df_train['pat_hosp_id'].unique()
test_pat=df_test['pat_hosp_id'].unique()
val_pat=df_val['pat_hosp_id'].unique()

df_train = df[df.index.get_level_values('pat_hosp_id').isin(train_pat)]
df_val = df[df.index.get_level_values('pat_hosp_id').isin(val_pat)]
df_test = df[df.index.get_level_values('pat_hosp_id').isin(test_pat)]
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 688151 entries, (818812, Timestamp('2012-01-23 00:00:00')) to (7171037, Timestamp('2008-04-11 00:00:00'))
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   level_0          688151 non-null  int64         
 1   pat_datetime     688151 non-null  datetime64[ns]
 2   pat_weight_act   688151 non-null  float16       
 3   mon_etco2        582366 non-null  float16       
 4   mon_hr           682682 non-null  float16       
 5   mon_ibp_mean     626468 non-null  float16       
 6   mon_rr           653734 non-null  float16       
 7   mon_sat          674427 non-null  float16       
 8   vent_m_fio2      683642 non-null  float16       
 9   vent_m_peep      674252 non-null  float16       
 10  vent_m_ppeak     687071 non-null  float16       
 11  vent_m_rr        688145 non-null  float16       
 12  vent_m_tv_exp    685418 non-null  float16       
 

In [5]:
def y_modelling(df):
    y = df[['Reintub', 'Adnum', 'idx_1hr']]
    y = y.groupby(['Adnum', 'idx_1hr'], sort=False).agg(['max'])
    y.columns = ["_".join(a) for a in y.columns.to_flat_index()]
    y.reset_index(drop=True, inplace=True, level='idx_1hr')
    y = y.reset_index().drop_duplicates().set_index(
        ['Adnum'])
    y = y[~y.index.duplicated(keep='last')]
    return y
# from dataframwe with label per minute, to array with label per admission

Y_TRAIN=y_modelling(df_train)
Y_TEST=y_modelling(df_test)
Y_VAL=y_modelling(df_val)
print(Y_TRAIN.value_counts())
print(Y_VAL.value_counts())
print(Y_TEST.value_counts())
Y_TRAIN=Y_TRAIN['Reintub_max'].to_numpy()
Y_TEST=Y_TEST['Reintub_max'].to_numpy()
Y_VAL=Y_VAL['Reintub_max'].to_numpy()
Y_TRAIN=np.append(Y_TRAIN,Y_VAL)


In [7]:
def x_modelling(df):
    temp = df[['Age', 'Adnum', 'idx_1hr','pat_weight_act','Diagnose','mis']]
    df = df.drop(['Age','pat_weight_act','Extubation_date','level_0','pat_datetime', 'Reintub','Diagnose','mis'], axis=1)
    print(df.info())
    df_temp=df
    df = df.groupby(['Adnum', 'idx_1hr'], sort=False).agg(['mean', 'std'])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df = df.stack().unstack([2, 1])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df = df.reset_index().drop_duplicates().set_index('Adnum')
    df = df.fillna(method='ffill').fillna(method='bfill')

    temp = temp.groupby('Adnum', sort=False).agg(['mean'])
    temp.columns = ["_".join(a) for a in temp.columns.to_flat_index()]
    temp = temp.reset_index().drop_duplicates().set_index('Adnum')
    temp = temp[~temp.index.duplicated(keep='last')]
    print(temp.info())

    df_temp = df_temp.drop('idx_1hr',axis=1)
    df_temp = df_temp.groupby('Adnum',sort=False).agg([lin_reg_coef])
    df_temp.columns = ["_".join(a) for a in df_temp.columns.to_flat_index()]
    df_temp = df_temp.reset_index().drop_duplicates().set_index('Adnum')
    df_temp = df_temp[~df_temp.index.duplicated(keep='last')]

    #df = df.merge(temp, right_index=True, left_index=True, how='left')
    df = df.merge(df_temp, right_index=True, left_index=True, how='left')
    del temp
    del df_temp
    return df


# Calculate features per admission per hour
X_TRAIN=x_modelling(memory_upscale(df_train))
X_TEST=x_modelling(memory_upscale(df_test))
X_VAL=x_modelling(memory_upscale(df_val))
print(X_TRAIN.info())
print(X_TEST.info())
X_TRAIN=X_TRAIN.fillna(value=0)
X_VAL=X_VAL.fillna(value=0)
X_TEST=X_TEST.fillna(value=0)

X_TRAIN=X_TRAIN.append(X_VAL)

c:\Users\berend\Documents\Python_Scripts\Patient_Selection\all_own_funct.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype('float64')
c:\Users\berend\Documents\Python_Scripts\Patient_Selection\all_own_funct.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype('int64')
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 688151 entries, (818812, Timestamp('2012-01-23 00:00:00')) to (7171037, Timestamp('2008-04-11 00:00:00'))
Data columns (total 

In [8]:
# Save the test and train set for later
f = open(os.path.join(output_folder, 'x_train.txt'), 'wb')
pickle.dump(X_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'x_test.txt'), 'wb')
pickle.dump(X_TEST, f)
f.close()

f = open(os.path.join(output_folder, 'y_train.txt'), 'wb')
pickle.dump(Y_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'y_test.txt'), 'wb')
pickle.dump(Y_TEST, f)
f.close()

In [9]:
# Optimizer set up for random forest
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_TRAIN, Y_TRAIN)




Fitting 3 folds for each of 20 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.5min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 14, 18, 22, 26, 30,
                                                      34, 38, 42, 46, 50,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [11]:
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix
# Evaluate the best and base model
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42,max_depth=10)
base_model.fit(X_TRAIN, Y_TRAIN)
base_accuracy = evaluate(base_model, X_TEST, Y_TEST,'base_accuracy RF',output_folder)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_TEST,Y_TEST,'Best_random RF',output_folder)
if base_accuracy > random_accuracy:
    best_random=base_model



Model Performance
average_precision = 0.06159421132252446
F1 score: 0.0851063829787234 
AUC = 0.3910867938645717
Model Performance
average_precision = 0.050300422668065725
F1 score: 0.0 
AUC = 0.34137673026561915


In [12]:
# Calculate Brier score loss, not used

"""
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from matplotlib import cm

# Gaussian Naive-Bayes with no calibration
# GaussianNB itself does not support sample-weights
prob_pos_clf = best_random.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(best_random, cv=2, method='isotonic')
clf_isotonic.fit(X_TRAIN, Y_TRAIN)
prob_pos_isotonic = clf_isotonic.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(best_random, cv=2, method='sigmoid')
clf_sigmoid.fit(X_TRAIN, Y_TRAIN)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X_TEST)[:, 1]

print("Brier score losses: (the smaller the better)")

clf_score = brier_score_loss(Y_TEST, prob_pos_clf)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(Y_TEST, prob_pos_isotonic)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(Y_TEST, prob_pos_sigmoid)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for RF on training set\n\n")
    file.write("Brier score losses: (the smaller the better)\n")
    file.write("No calibration: %1.3f" % clf_score)
    file.write("With isotonic calibration: %1.3f" % clf_isotonic_score)
    file.write("With sigmoid calibration: %1.3f" % clf_sigmoid_score)
"""

Brier score losses: (the smaller the better)
No calibration: 0.120
With isotonic calibration: 0.097
With sigmoid calibration: 0.087


In [13]:
# Create AUROC plot, confusion matrix and other results for Random Forest

from all_own_funct import roc_auc_ci
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures_alls.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures_all.pdf"))
    os.remove(os.path.join(output_folder,f"Result_scores_all.txt"))
clf = best_random 


y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

# write scoring metrics to file
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for RF on training set\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of random forrest")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)

n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write("\nConfidence interval for the score: [{:0.3f} - {:0.3}]\n".format(
    confidence_lower, confidence_upper))
a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)


plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)

plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Random Forrest hour data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)



(0.2889637409070993, 0.49320984682204405)


In [14]:
# Hyperparameters for Logistic Regression
# The solvers
solver =['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# Class weight optimizer
class_weight=[{0:0.1,1:0.9},'balanced',None]
# Penalty
penalty=['l1', 'l2', 'elasticnet', None]
# Inverse of regularization strength
C=np.logspace(-3,3,7)
# Bootsrapping
bootstrap = [True, False]
# Create the random grid
random_grid = {'solver':solver,
                'class_weight':class_weight,
                'penalty':penalty,
                'C':C,
                'penalty':penalty}

# First create the base model to tune
lr = LogisticRegression(max_iter=1000)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
lr_random = RandomizedSearchCV(estimator = lr , param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
lr_random.fit(X_TRAIN, Y_TRAIN)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  45 out of  60 | elapsed:    8.4s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   22.8s finished


RandomizedSearchCV(cv=3, estimator=LogisticRegression(max_iter=1000), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                                        'class_weight': [{0: 0.1, 1: 0.9},
                                                         'balanced'],
                                        'penalty': ['l1', 'l2', 'elasticnet'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   random_state=42, verbose=2)

In [15]:
# Evaluate the base and optimized model
base_model_lr = LogisticRegression(max_iter=1000)
base_model_lr.fit(X_TRAIN, Y_TRAIN)
base_accuracy_lr = evaluate(base_model_lr, X_TEST, Y_TEST,'base LR',output_folder)

best_random_lr = lr_random.best_estimator_
random_accuracy_lr = evaluate(best_random_lr, X_TEST,Y_TEST,'Best Random LR',output_folder)

if base_accuracy_lr > random_accuracy_lr:
    best_random_lr=base_model_lr

# Brier scores
"""
# Gaussian Naive-Bayes with no calibration
# GaussianNB itself does not support sample-weights
prob_pos_clf = best_random_lr.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(best_random_lr, cv=2, method='isotonic')
clf_isotonic.fit(X_TRAIN, Y_TRAIN)
prob_pos_isotonic = clf_isotonic.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(best_random_lr, cv=2, method='sigmoid')
clf_sigmoid.fit(X_TRAIN, Y_TRAIN)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X_TEST)[:, 1]

print("Brier score losses: (the smaller the better)")

clf_score = brier_score_loss(Y_TEST, prob_pos_clf)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(Y_TEST, prob_pos_isotonic)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(Y_TEST, prob_pos_sigmoid)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for LR on training set\n\n")
    file.write("Brier score losses: (the smaller the better)\n")
    file.write("No calibration: %1.3f" % clf_score)
    file.write("With isotonic calibration: %1.3f" % clf_isotonic_score)
    file.write("With sigmoid calibration: %1.3f" % clf_sigmoid_score)
"""

Model Performance
average_precision = 0.12384697904612935
F1 score: 0.1523809523809524 
AUC = 0.684156378600823
Model Performance
average_precision = 0.10820045734044251
F1 score: 0.0 
AUC = 0.680976430976431
Brier score losses: (the smaller the better)
No calibration: 0.167
With isotonic calibration: 0.059
With sigmoid calibration: 0.059


In [16]:
# Create AUROC plot, confusion matrix and other results for logistic regression

clf = best_random_lr


y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for LR on training\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of logistic regression")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)


n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write("\nConfidence interval for the score: [{:0.3f} - {:0.3}]\n".format(
    confidence_lower, confidence_upper))

a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)

plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)


plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Logistic Regression data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)
pdf.close()


(0.5700501130413288, 0.7982626441603172)


In [17]:
# Save the best models
import pickle
f = open(os.path.join(output_folder,'ran_for.sav'), 'wb')
pickle.dump(best_random, f)
f.close()

f = open(os.path.join(output_folder,'log_reg.sav'), 'wb')
pickle.dump(best_random_lr, f)
f.close()